In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.style import use as usestyle
from matplotlib.colors import BoundaryNorm
from datetime import datetime, timedelta
from os import path, mkdir
from importlib import reload
import makevids as mkv
from astropy.convolution import convolve
from glob import glob
from time import perf_counter
from concurrent.futures import ProcessPoolExecutor
# import pyvista as pv

In [ ]:
# forestflux_wetsoil = 

In [ ]:
from matplotlib import font_manager as fm
fontdir = "/home/ascheb/libfonts/*.ttf"
for fpath in glob(fontdir):
    fm.fontManager.addfont(fpath)
usestyle("/home/ascheb/paperplots.mplstyle")
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'custom'
rcParams['mathtext.rm'] = 'Liberation Serif'
rcParams['mathtext.it'] = 'Liberation Serif'
rcParams['mathtext.bf'] = 'Liberation Serif'

In [ ]:
def calc_mse(datasub):
    return((1/(1+datasub["VaporMix"]/1000))*datasub["Temperature"]+
           (datasub["VaporMix"]/1000/(1+datasub["VaporMix"]/1000))*(1864/1004)+
           (datasub["VaporMix"]/1000/(1+datasub["VaporMix"]/1000))*(2.5*10**6/1004)+
           9.81*datasub["z"]/1004)

## Load Model Data from End of the Simulation for Precip Comparison

In [ ]:
ftime = "2022-06-18-000000"
afile_pasture_end = xr.open_dataset(f"/moonbow/ascheb/idealized/unipasture-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_suburb_end = xr.open_dataset(f"/moonbow/ascheb/idealized/unisuburb-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_coniforest_end = xr.open_dataset(f"/moonbow/ascheb/idealized/coniforest-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_pastureconiforest_end = xr.open_dataset(f"/moonbow/ascheb/idealized/pastureconiforest-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_suburbconiforest_end = xr.open_dataset(f"/moonbow/ascheb/idealized/suburbconiforest-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_suburbpasture_end = xr.open_dataset(f"/moonbow/ascheb/idealized/suburbpasture-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")

## Plot of Model Domains

In [ ]:
fig, ((ax11, ax12, ax13), (ax21, ax22, ax23)) = plt.subplots(2, 3, figsize = (4,5), dpi = 200, layout = "compressed")
axlabels = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)"]
for i, ax in enumerate(fig.get_axes()):
    ax.set_aspect(1)
    ax.set_yticks([0, 300, 800], labels = [-150, "0  ", 250])
    ax.set_xticks([0, 100, 200, 300, 400], labels = [-100, -50, 0, 50, 100])
    left, width = 0, 0.16
    bottom, height = 0.92, 0.08
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 6.5, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
for ax in (ax11, ax21):
    ax.set_ylabel("Distance from Shore (km)")
    ax.tick_params(axis = "x", bottom = False, labelbottom = False, left = True, labelleft = True)
ax22.set_xlabel("Distance from Center (km)")
for ax in (ax12, ax13, ax23):
    ax.tick_params(axis = "both", bottom = False, left = False, labelbottom = False, labeltop = False, labelleft = False)
ax22.tick_params(axis = "y", left = False, labelleft = False, bottom = True, labelbottom = True)
# for ax in (ax21, ax22, ax23):
#     forestbounds = ax.plot([100, 100], [300, 800], color = "black", linestyle = "--", label = "Stripe Boundaries", linewidth = 0.5)
#     ax.plot([300, 300], [300, 800], color = "black", linestyle = "--", linewidth = 0.5)
# for ax in fig.get_axes():
#     shoreline = ax.axhline(301, color = "white", linestyle = "--", label = "Shoreline", linewidth = 0.5)
#     ax.legend(loc  = "lower left", framealpha = 0, fontsize = 6.5, labelcolor = "white", bbox_to_anchor = (0, 0), borderaxespad = 0, borderpad = 0.1)
ax11.set_title("Uniform Pasture (UP)", fontfamily = "Liberation Serif", y = 1.02)
ax12.set_title("Uniform Forest (UF)", fontfamily = "Liberation Serif", y = 1.02)
ax13.set_title("Uniform Suburb (US)", fontfamily = "Liberation Serif", y = 1.02)
ax21.set_title("Pasture-Forest (PF)", fontfamily = "Liberation Serif", y = 1.02)
ax22.set_title("Suburb-Forest (SF)", fontfamily = "Liberation Serif", y = 1.02)
ax23.set_title("Suburb-Pasture (SP)", fontfamily = "Liberation Serif", y = 1.02)
fig.get_layout_engine().set(h_pad = 0.05, hspace = 0.1)

ax11.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_pasture_end["VegClass"][1,:,:].where(afile_pasture_end["PatchArea"][1,:,:]== 1), colors = "gold")
ax11.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_pasture_end["VegClass"][0,:,:].where(afile_pasture_end["PatchArea"][0,:,:]== 1), colors = "navy")

ax11.annotate("OCEAN", (201, 100), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
ax11.annotate("PASTURE", (201, 500), color = "black", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
# ax11.set_title("Uniform Pasture", fontsize = 20, fontweight = "demibold", fontfamily = "Liberation Serif")

ax12.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_coniforest_end["VegClass"][1,:,:].where(afile_coniforest_end["PatchArea"][1,:,:]== 1), colors = "darkgreen")
ax12.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_coniforest_end["VegClass"][0,:,:].where(afile_coniforest_end["PatchArea"][0,:,:]== 1), colors = "navy")
# sfcds["PatchArea_AREA"][1,:,:].plot()
ax12.annotate("OCEAN", (201, 100), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
ax12.annotate("CONIFEROUS \n FOREST", (201, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 8, horizontalalignment = "center", verticalalignment = "center")
# ax12.set_title("Uniform Coniferous Forest", fontsize = 20, fontweight = "demibold", fontfamily = "Liberation Serif")

ax13.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_suburb_end["VegClass"][1,:,:].where(afile_suburb_end["PatchArea"][1,:,:]== 1), colors = "maroon")
ax13.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_suburb_end["VegClass"][0,:,:].where(afile_suburb_end["PatchArea"][0,:,:]== 1), colors = "navy")
# sfcds["PatchArea_AREA"][1,:,:].plot()
ax13.annotate("OCEAN", (201, 100), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
ax13.annotate("SUBURB", (201, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
# ax13.set_title("Uniform Suburb", fontsize = 20, fontweight = "demibold", fontfamily = "Liberation Serif")

ax21.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_pastureconiforest_end["VegClass"][1,:,:].where(afile_pastureconiforest_end["PatchArea"][1,:,:]==1), colors = ["darkgreen", "gold"])
ax21.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_pastureconiforest_end["VegClass"][0,:,:].where(afile_pastureconiforest_end["PatchArea"][0,:,:]==1), colors = "navy")
ax21.annotate("OCEAN", (201, 100), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
ax21.annotate("PAS-\nTURE", (51, 500), color = "black", fontfamily = "sans", fontweight = "bold", fontsize = 5, horizontalalignment = "center", verticalalignment = "center")
ax21.annotate("PAS-\nTURE", (351, 500), color = "black", fontfamily = "sans", fontweight = "bold", fontsize = 5, horizontalalignment = "center", verticalalignment = "center")
ax21.annotate("CONIFEROUS \n FOREST", (201, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 4, horizontalalignment = "center", verticalalignment = "center")
# ax21.set_title("Pasture-Forest", fontsize = 20, fontweight = "demibold", fontfamily = "Liberation Serif")

ax22.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_suburbconiforest_end["VegClass"][1,:,:].where(afile_suburbconiforest_end["PatchArea"][1,:,:]==1), colors = ["darkgreen", "maroon"])
ax22.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_suburbconiforest_end["VegClass"][0,:,:].where(afile_suburbconiforest_end["PatchArea"][0,:,:]==1), colors = "navy")
ax22.annotate("OCEAN", (201, 100), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
ax22.annotate("SUB-\nURB", (51, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 5, horizontalalignment = "center", verticalalignment = "center")
ax22.annotate("SUB-\nURB", (351, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 5, horizontalalignment = "center", verticalalignment = "center")
ax22.annotate("CONIFEROUS \n FOREST", (201, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 4, horizontalalignment = "center", verticalalignment = "center")
# ax22.set_title("Suburb-Forest", fontsize = 20, fontweight = "demibold", fontfamily = "Liberation Serif")

ax23.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_suburbpasture_end["VegClass"][1,:,:].where(afile_suburbpasture_end["PatchArea"][1,:,:]==1), colors = ["gold", "maroon"])
ax23.contourf(np.linspace(0, 400, 400), np.linspace(0, 800, 800), afile_suburbpasture_end["VegClass"][0,:,:].where(afile_suburbpasture_end["PatchArea"][0,:,:]==1), colors = "navy")
ax23.annotate("OCEAN", (201, 100), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 10, horizontalalignment = "center", verticalalignment = "center")
ax23.annotate("SUB-\nURB", (51, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 5, horizontalalignment = "center", verticalalignment = "center")
ax23.annotate("SUB-\nURB", (351, 500), color = "white", fontfamily = "sans", fontweight = "bold", fontsize = 5, horizontalalignment = "center", verticalalignment = "center")
ax23.annotate("PASTURE", (201, 500), color = "black", fontfamily = "sans", fontweight = "bold", fontsize = 5, horizontalalignment = "center", verticalalignment = "center")
# ax23.set_title("Suburb-Pasture", fontsize = 15, fontweight = "demibold", fontfamily = "Liberation Serif")

fig.suptitle("Simulation Domains", fontweight = "demibold", fontfamily = "Liberation Serif")
fig.savefig("/moonbow/ascheb/idealized/PaperFigs/domaincomp.png")

In [ ]:
print(afile_pasture.data_vars)

In [ ]:
# import pandas as pd
# vegpropsdf = pd.DataFrame(columns = ["VegHeight", "RoughnessLength", "LAI", "VegFrac", "VegAlbedo", "TotalAlbedo"])
# vegpropsdf["TotalAlbedo"] = np.array([afile_pasture["SurfaceAlbedo"].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_suburb["SurfaceAlbedo"].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_coniforest["SurfaceAlbedo"].where(afile_pasture["Patch"][1,:,:]==1).mean()])
# vegpropsdf["VegHeight"] = np.array([afile_pasture["VegHeight"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_suburb["VegHeight"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_coniforest["VegHeight"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean()])
# vegpropsdf["RoughnessLength"] = np.array([afile_pasture["SurfaceRoughness"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_suburb["SurfaceRoughness"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_coniforest["SurfaceRoughness"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean()])
# vegpropsdf["LAI"] = np.array([afile_pasture["LeafArea"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_suburb["LeafArea"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_coniforest["LeafArea"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean()])
# vegpropsdf["VegFrac"] = np.array([afile_pasture["VegFraction"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_suburb["VegFraction"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_coniforest["VegFraction"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean()])
# vegpropsdf["VegAlbedo"] = np.array([afile_pasture["VegAlbedo"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_suburb["VegAlbedo"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean(), afile_coniforest["VegAlbedo"][1,:,:].where(afile_pasture["Patch"][1,:,:]==1).mean()])

#print(vegpropsdf)
#vegpropsdf.to_csv("vegprops.csv")

In [ ]:
print(afile_pasture["z"])

## Domain-Total Precip Comparison

In [ ]:
rams_areafactor = 2.5*10**5
landarea = 500*400*rams_areafactor
upprecip = afile_pasture_end["RainPrecipTotal"].sum()*rams_areafactor/landarea #all precip falls over land in these simulations
print("UP Simulation Mean Precipitation", np.round(upprecip.values, 3))
usprecip = afile_suburb_end["RainPrecipTotal"].sum()*rams_areafactor/landarea #all precip falls over land in these simulations
print("US Simulation Mean Precipitation", np.round(usprecip.values, 3))
ucfprecip = afile_coniforest_end["RainPrecipTotal"].sum()*rams_areafactor/landarea #all precip falls over land in these simulations
print("UCF Simulation Mean Precipitation", np.round(ucfprecip.values, 3))
pcfprecip = afile_pastureconiforest_end["RainPrecipTotal"].sum()*rams_areafactor/landarea #all precip falls over land in these simulations
print("PCF Simulation Mean Precipitation", np.round(pcfprecip.values, 3))
scfprecip = afile_suburbconiforest_end["RainPrecipTotal"].sum()*rams_areafactor/landarea #all precip falls over land in these simulations
print("SCF Simulation Mean Precipitation", np.round(scfprecip.values, 3))
spprecip = afile_suburbpasture_end["RainPrecipTotal"].sum()*rams_areafactor/landarea #all precip falls over land in these simulations
print("SP Simulation Mean Precipitation", np.round(spprecip.values, 3))

## Comparison of Precip That Falls over Different Land Surfaces

In [ ]:
pcf_forestprecip = afile_pastureconiforest_end["RainPrecipTotal"].where(afile_pastureconiforest_end["VegClass"].isel(patch = 1) == 4).where(afile_pastureconiforest_end["PatchArea"].isel(patch = 1) == 1).mean().values
print("PCF Simulation Forest Precipitation", np.round(pcf_forestprecip, 3))
pcf_pastureprecip = afile_pastureconiforest_end["RainPrecipTotal"].where(afile_pastureconiforest_end["VegClass"].isel(patch = 1) == 15).where(afile_pastureconiforest_end["PatchArea"].isel(patch = 1) == 1).mean().values
print("PCF Simulation Pasture Precipitation", np.round(pcf_pastureprecip, 3))
scf_forestprecip = afile_suburbconiforest_end["RainPrecipTotal"].where(afile_suburbconiforest_end["VegClass"].isel(patch = 1) == 4).where(afile_suburbconiforest_end["PatchArea"].isel(patch = 1) == 1).mean().values
print("SCF Simulation Forest Precipitation", np.round(scf_forestprecip, 3))
scf_suburbprecip = afile_suburbconiforest_end["RainPrecipTotal"].where(afile_suburbconiforest_end["VegClass"].isel(patch = 1) == 19).where(afile_suburbconiforest_end["PatchArea"].isel(patch = 1) == 1).mean().values
print("SCF Simulation Suburb Precipitation", np.round(scf_suburbprecip, 3))
sp_pastureprecip = afile_suburbpasture_end["RainPrecipTotal"].where(afile_suburbpasture_end["VegClass"].isel(patch = 1) == 15).where(afile_suburbpasture_end["PatchArea"].isel(patch = 1) == 1).mean().values
print("SP Simulation Pasture Precipitation", np.round(sp_pastureprecip, 3))
sp_suburbprecip = afile_suburbpasture_end["RainPrecipTotal"].where(afile_suburbpasture_end["VegClass"].isel(patch = 1) == 19).where(afile_suburbpasture_end["PatchArea"].isel(patch = 1) == 1).mean().values
print("SP Simulation Suburb Precipitation", np.round(sp_suburbprecip, 3))

## Six-Panel Plan View of Total Precipitation

In [ ]:
t = datetime.strptime(ftime, "%Y-%m-%d-%H%M%S")
folderprepath = "/moonbow/ascheb/idealized/"
fig, ((ax11, ax12, ax13), (ax21, ax22, ax23)) = plt.subplots(2, 3, figsize = (4, 5), dpi = 200, layout = "compressed")
axlabels = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)"]
for i, ax in enumerate(fig.get_axes()):
    ax.set_aspect(1)
    ax.set_yticks([200, 300, 450, 600, 800], labels = ["-50 ", "0  ", "75 ", 150, 250])
    ax.set_xticks([0, 100, 200, 300, 400], labels = [-100, -50, 0, 50, 100])
    left, width = 0, 0.16
    bottom, height = 0.92, 0.08
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "whitesmoke")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 6.5, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
for ax in (ax11, ax21):
    ax.set_ylabel("Distance from Shore (km)")
    ax.tick_params(axis = "x", bottom = False, labelbottom = False, left = True, labelleft = True)
ax22.set_xlabel("Distance from Center (km)")
for ax in (ax12, ax13, ax23):
    ax.tick_params(axis = "both", bottom = False, left = False, labelbottom = False, labeltop = False, labelleft = False)
ax22.tick_params(axis = "y", left = False, labelleft = False, bottom = True, labelbottom = True)
# for ax in (ax21, ax22, ax23):
#     forestbounds = ax.plot([100, 100], [300, 800], color = "black", linestyle = "--", label = "Stripe Boundaries", linewidth = 0.5)
#     ax.plot([300, 300], [300, 800], color = "black", linestyle = "--", linewidth = 0.5)
# for ax in fig.get_axes():
#     shoreline = ax.axhline(301, color = "white", linestyle = "--", label = "Shoreline", linewidth = 0.5)
#     ax.legend(loc  = "lower left", framealpha = 0, fontsize = 6.5, labelcolor = "white", bbox_to_anchor = (0, 0), borderaxespad = 0, borderpad = 0.1)
ax11.set_title("Uniform Pasture (UP)", fontfamily = "Liberation Serif", y = 1.02)
ax12.set_title("Uniform Forest (UF)", fontfamily = "Liberation Serif", y = 1.02)
ax13.set_title("Uniform Suburb (US)", fontfamily = "Liberation Serif", y = 1.02)
ax21.set_title("Pasture-Forest (PF)", fontfamily = "Liberation Serif", y = 1.02)
ax22.set_title("Suburb-Forest (SF)", fontfamily = "Liberation Serif", y = 1.02)
ax23.set_title("Suburb-Pasture (SP)", fontfamily = "Liberation Serif", y = 1.02)
fig.get_layout_engine().set(h_pad = 0.05, hspace = 0.1)
        
for ax in (ax21, ax22, ax23):
    forestbounds = ax.plot([100, 100], [300, 800], color = "black", linestyle = "--", label = "Stripe Boundaries", linewidth = 0.5)
    ax.plot([300, 300], [300, 800], color = "black", linestyle = "--", linewidth = 0.5)
for ax in fig.get_axes():
    shoreline = ax.axhline(301, color = "black", linestyle = "-.", label = "Shoreline", linewidth = 0.5)
    ax.legend(loc  = "lower left", framealpha = 0, fontsize = 6.5, labelcolor = "black", bbox_to_anchor = (0, 0), borderaxespad = 0, borderpad = 0.1)

fig.suptitle(f"Total Rainfall at {str((t - timedelta(hours = 5)).hour).zfill(2)}{str((t - timedelta(hours = 5)).minute).zfill(2)} LT", fontfamily = "Liberation Serif")
rainbounds = [0, 0.1, 0.3, 0.5, 1.5, 2.5, 5.5, 7.5, 11.75, 15, 20, 25, 37.5, 50, 70]
rainnorm = BoundaryNorm(boundaries = rainbounds, ncolors = 256, extend = "max")
# rainnorm = BoundaryNorm(boundaries = [0.1, 2.5, 5, 7.5, 10, 12.5, 15, 17.5, 20, 22.5, 25, 27.5, 30], ncolors = 256, extend = "max")
rainmp = ax11.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), (afile_pasture_end["RainPrecipTotal"].where(afile_pasture_end["RainPrecipTotal"]>0.01)).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
ax12.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), (afile_coniforest_end["RainPrecipTotal"].where(afile_coniforest_end["RainPrecipTotal"]>0.01)).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
ax13.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), (afile_suburb_end["RainPrecipTotal"].where(afile_suburb_end["RainPrecipTotal"]>0.01)).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
ax21.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), (afile_pastureconiforest_end["RainPrecipTotal"].where(afile_pastureconiforest_end["RainPrecipTotal"]>0.01)).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
ax22.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), (afile_suburbconiforest_end["RainPrecipTotal"].where(afile_suburbconiforest_end["RainPrecipTotal"]>0.01)).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
ax23.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), (afile_suburbpasture_end["RainPrecipTotal"].where(afile_suburbpasture_end["RainPrecipTotal"]>0.01)).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
cbar = fig.colorbar(rainmp, ax = fig.get_axes(), orientation = "horizontal", fraction = 0.05, extend = "max"); cbar.set_label("Total Rainfall (mm)", fontfamily = "Liberation Serif")
fig.savefig(f"{folderprepath}PaperFigs/raintotal_{str(t.hour).zfill(2)}{str(t.minute).zfill(2)}z.png")
plt.close()
del ax11; del ax12; del ax13
del ax21; del ax22; del ax23
del fig; del cbar

## Close Files from the End of the Simulation

In [ ]:
afile_pasture_end.close(); del afile_pasture_end
afile_coniforest_end.close(); del afile_coniforest_end
afile_suburb_end.close(); del afile_suburb_end
afile_pastureconiforest_end.close(); del afile_pastureconiforest_end
afile_suburbconiforest_end.close(); del afile_suburbconiforest_end
afile_suburbpasture_end.close(); del afile_suburbpasture_end

## Load Files from 1700z (1200 LT) for Rest of Analysis

In [ ]:
noontime = "2022-06-17-170000"
afile_pasture_noon = xr.open_dataset(f"/moonbow/ascheb/idealized/unipasture-wind/ppfiles_paper/mvars-cart-{noontime}-g1.nc")
afile_suburb_noon = xr.open_dataset(f"/moonbow/ascheb/idealized/unisuburb-wind/ppfiles_paper/mvars-cart-{noontime}-g1.nc")
afile_coniforest_noon = xr.open_dataset(f"/moonbow/ascheb/idealized/coniforest-wind/ppfiles_paper/mvars-cart-{noontime}-g1.nc")
afile_pastureconiforest_noon = xr.open_dataset(f"/moonbow/ascheb/idealized/pastureconiforest-wind/ppfiles_paper/mvars-cart-{noontime}-g1.nc")
afile_suburbconiforest_noon = xr.open_dataset(f"/moonbow/ascheb/idealized/suburbconiforest-wind/ppfiles_paper/mvars-cart-{noontime}-g1.nc")
afile_suburbpasture_noon = xr.open_dataset(f"/moonbow/ascheb/idealized/suburbpasture-wind/ppfiles_paper/mvars-cart-{noontime}-g1.nc")

In [ ]:
# print(afile_pasture

In [ ]:
print(afile_pastureconiforest_noon["VertIntLiq"])
print(afile_suburbpasture_noon["VertIntIce"])
print(afile_suburbpasture_noon["VertIntCond"])

In [ ]:
# print([var for var in afile_pastureconiforest_noon.variables])

## Plot Skew-T Comparison Between Pasture and Forest in PF Simulation

In [ ]:
forestthermo = afile_pastureconiforest_noon[["Pressure", "Temperature", "Dewpoint", "VaporMix"]].isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("y", "x"))
pasturethermo = (0.5*afile_pastureconiforest_noon[["Pressure", "Temperature", "Dewpoint", "VaporMix"]].isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("y", "x"))+0.5*afile_pastureconiforest_noon[["Pressure", "Temperature", "Dewpoint", "VaporMix"]].isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("y", "x")))

import metpy.calc as mcalc
from metpy.plots import SkewT
from metpy.units import units

tfig = plt.figure(figsize = (5, 4), dpi = 200)
testskew = SkewT(tfig, rotation = 45)
testskew.ax.set_ylim(1050, 600)
testskew.ax.set_xlim(10, 40)
# testskew.ax.axhline(forestthermo["Pressure"].sel(z = 200, method = "nearest"), color = "black")
fortemp, = testskew.plot(forestthermo["Pressure"]*units.hPa, forestthermo["Temperature"]*units.K-273.15*units.K, color = "red", label = "Temperature over Forest", linewidth = 1)
pastemp, = testskew.plot(pasturethermo["Pressure"]*units.hPa, pasturethermo["Temperature"]*units.K-273.15*units.K, color = "maroon", label = "Temperature over Pasture", linewidth = 1)
fordew, = testskew.plot(forestthermo["Pressure"]*units.hPa, forestthermo["Dewpoint"]*units.K-273.15*units.K, color = "dodgerblue", label = "Dewpoint over Forest", linewidth = 1)
pasdew, = testskew.plot(pasturethermo["Pressure"]*units.hPa, pasturethermo["Dewpoint"]*units.K-273.15*units.K, color = "midnightblue", label = "Dewpoint over Pasture", linewidth = 1)
dryads = testskew.plot_dry_adiabats(t0 = np.arange(-40, 200, 10)*units.degC, colors = "tomato", linewidths = 0.75, label = "Dry Adiabats", alpha = 0.5)
moistads = testskew.plot_moist_adiabats(t0 = np.arange(-40, 45, 5)*units.degC, colors = "dodgerblue", linewidths = 0.75, label = "Moist Adiabats", alpha = 0.5)
# testskew.plot_mixing_lines(pressure = np.arange(1025, 450, -25)*units.hPa, colors = "limegreen", linewidths = 0.75, label = ")
proflegend = testskew.ax.legend(framealpha = 0, fontsize = 10, loc = "upper right", handles = [fortemp, pastemp, fordew, pasdew], bbox_to_anchor = (1.02, 1.03))
testskew.ax.add_artist(proflegend)
backlegend = testskew.ax.legend(framealpha = 0, fontsize = 10, loc = "lower left", handles = [dryads, moistads], bbox_to_anchor=(0.0, -0.01))
testskew.ax.tick_params(labelsize = 9)
testskew.ax.add_artist(backlegend)
testskew.ax.set_ylabel("Pressure (hPa)", fontsize = 12)
testskew.ax.set_xlabel("Temperature (C)", fontsize = 12)
testskew.ax.set_title("Forest and Pasture Thermodynamic Profiles at 1200 LT", fontsize = 12)
plt.savefig("/moonbow/ascheb/idealized/PaperFigs/idealized_skewtcomp_1700z.png")

## Plot Moist Static Energy profiles for uniform simulations

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (3,3), dpi = 200, layout = "compressed")
ax.set_title("Moist Static Energy at 1200 LT", fontsize = 10)
pasturemse = calc_mse(afile_pasture_noon.isel(y = slice(500,800)).sel(z=slice(0,3000)).mean(dim = ("x", "y")))
forestmse = calc_mse(afile_coniforest_noon.isel(y = slice(500,800)).sel(z=slice(0,3000)).mean(dim = ("x", "y")))
suburbmse = calc_mse(afile_suburb_noon.isel(y = slice(500,800)).sel(z=slice(0,3000)).mean(dim = ("x", "y")))
# ax.plot(afile_pasture_noon["Temperature"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y")) +
#           2.5*10**6*afile_pasture_noon["VaporMix"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y"))*10**(-3)*(1/1004) +
#           9.81/1004 * afile_pasture_noon["z"].sel(z = slice(0, 3000)), 
#           afile_pasture_noon["z"].sel(z = slice(0, 3000)), color = "red", linewidth = 1, zorder = 1, linestyle = "--", label = "Uniform Pasture (UP)")
# ax.plot(afile_coniforest_noon["Temperature"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y")) +
#           2.5*10**6*afile_coniforest_noon["VaporMix"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y"))*10**(-3)*(1/1004) +
#           9.81/1004 * afile_coniforest_noon["z"].sel(z = slice(0, 3000)), 
#           afile_coniforest_noon["z"].sel(z = slice(0, 3000)), color = "darkgreen", linewidth = 1, zorder = 2, label = "Uniform Forest (UF)")
# ax.plot(afile_suburb_noon["Temperature"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y")) +
#           2.5*10**6*afile_suburb_noon["VaporMix"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y"))*10**(-3)*(1/1004) +
#           9.81/1004 * afile_suburb_noon["z"].sel(z = slice(0, 3000)), 
#           afile_suburb_noon["z"].sel(z = slice(0, 3000)), color = "dodgerblue", linewidth = 1, zorder = 0, label = "Uniform Suburb (US)")
ax.plot(pasturemse, afile_pasture_noon["z"].sel(z = slice(0, 3000)), color = "red", linewidth = 1, zorder = 0, label = "Uniform Pasture (UP)")
ax.plot(forestmse, afile_pasture_noon["z"].sel(z = slice(0,3000)), color = "dodgerblue", linewidth = 1, zorder = 2, label = "Uniform Forest (UF)")
ax.plot(suburbmse, afile_suburb_noon["z"].sel(z = slice(0,3000)), color = "black", linewidth = 1, zorder = 1, linestyle = "--", label = "Uniform Suburb (US)") 
ax.legend()
ax.set_ylabel("Altitude (m)", fontsize = 10)
ax.set_xlabel("Normalized Moist Static Energy (K)", fontsize = 10)
fig.savefig("/moonbow/ascheb/idealized/PaperFigs/MSE_unif.png")

### MSE Plots for Striped Simulations

In [ ]:
forestthermo = afile_pastureconiforest_noon[["Pressure", "Temperature", "Dewpoint", "VaporMix"]].isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("y", "x"))
pasturethermo = (0.5*afile_pastureconiforest_noon[["Pressure", "Temperature", "Dewpoint", "VaporMix"]].isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("y", "x"))+0.5*afile_pastureconiforest_noon[["Pressure", "Temperature", "Dewpoint", "VaporMix"]].isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("y", "x")))

fig, ax = plt.subplots(1, 1, figsize = (3,3), dpi = 200, layout = "compressed")
ax.set_title("Moist Static Energy at 1200 LT", fontsize = 10)
forest_stripe_mse = calc_mse(forestthermo.sel(z = slice(0,3000)))
pasture_stripe_mse = calc_mse(pasturethermo.sel(z = slice(0,3000)))
ax.plot(pasturemse, afile_pasture_noon["z"].sel(z = slice(0,3000)), color = "red", zorder = 0, linewidth = 1, label = "Uniform Pasture")
ax.plot(pasture_stripe_mse, afile_pasture_noon["z"].sel(z = slice(0,3000)), color = "red", linestyle = "--", linewidth = 1, zorder = 1, label = "PF Pasture Stripe")
ax.plot(forestmse, afile_pasture_noon["z"].sel(z = slice(0,3000)), color = "dodgerblue", linewidth = 1, zorder = 2, label = "Uniform Forest")
ax.plot(forest_stripe_mse, afile_pasture_noon["z"].sel(z = slice(0,3000)), color = "dodgerblue", linewidth = 1, linestyle = "--", zorder = 3, label = "PF Forest Stripe")
# ax.plot(afile_pasture_noon["Temperature"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y")) +
#           2.5*10**6*afile_pasture_noon["VaporMix"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y"))*10**(-3)*(1/1004) +
#           9.81/1004 * afile_pasture_noon["z"].sel(z = slice(0, 3000)), 
#           afile_pasture_noon["z"].sel(z = slice(0, 3000)), color = "red", linewidth = 1, zorder = 1, linestyle = "-", label = "Uniform Pasture")
# ax.plot(pasturethermo["Temperature"].sel(z = slice(0, 3000)) +
#           2.5*10**6*pasturethermo["VaporMix"].sel(z = slice(0,3000))*10**(-3)*(1/1004) +
#           9.81/1004 * pasturethermo["z"].sel(z = slice(0, 3000)), 
#           pasturethermo["z"].sel(z = slice(0, 3000)), color = "red", linewidth = 1, zorder = 1, linestyle = "--", label = "Pasture Stripe")
# ax.plot(afile_coniforest_noon["Temperature"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y")) +
#           2.5*10**6*afile_coniforest_noon["VaporMix"].isel(y = slice(500, 800)).sel(z = slice(0, 3000)).mean(dim = ("x", "y"))*10**(-3)*(1/1004) +
#           9.81/1004 * afile_coniforest_noon["z"].sel(z = slice(0, 3000)), 
#           afile_coniforest_noon["z"].sel(z = slice(0, 3000)), color = "dodgerblue", linewidth = 1, zorder = 2, label = "Uniform Forest (UF)")
# ax.plot((1/(1+forestthermo["VaporMix"]/1000) * forestthermo["Temperature"] + (forestthermo["VaporMix"]/1000/(1+forestthermo["VaporMix"]/1000)) * (1864/1004) + (forestthermo["VaporMix"]/1000/(1+forestthermo["VaporMix"]/1000)) * 2.5*10**6/1004 + forestthermo["z"] * 9.81/1004).sel(z = slice(0,3000)), forestthermo["z"].sel(z = slice(0,3000)), color = "red", linewidth = 1, zorder = 0, label = "'Exact'")
# ax.plot(forestthermo["Temperature"].sel(z = slice(0, 3000)) +
#           2.5*10**6*forestthermo["VaporMix"].sel(z = slice(0,3000))*10**(-3)*(1/1004) +
#           9.81/1004 * forestthermo["z"].sel(z = slice(0, 3000)), 
#           forestthermo["z"].sel(z = slice(0, 3000)), color = "dodgerblue", linewidth = 1, zorder = 1, linestyle = "--", label = "Approximation")
ax.legend()
ax.set_ylabel("Altitude (m)", fontsize = 10)
ax.set_xlabel("Normalized Moist Static Energy (K)", fontsize = 10)
fig.savefig("/moonbow/ascheb/idealized/PaperFigs/MSE_striped.png")

## Plot Difference Between Pasture and Forest Temps and Mixing Ratio

In [ ]:
forestthermo["VaporMix"] = afile_pastureconiforest_noon["VaporMix"].isel(x = slice(150, 250), y = slice(500,800)).mean(dim = ("x", "y"))
pasturethermo["VaporMix"] = 0.5*afile_pastureconiforest_noon["VaporMix"].isel(x = slice(0, 50), y = slice(500,800)).mean(dim = ("x", "y"))+0.5*afile_pastureconiforest_noon["VaporMix"].isel(x = slice(350, 400), y = slice(500,800)).mean(dim = ("x", "y"))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (4, 2), dpi = 600)
fig.suptitle("Difference in Temperature and Vapor Mixing Ratio \n Between Forest and Pasture at 1200 LT")
for ax in (ax1, ax2):
    ax.axhline(200, color = "black", linestyle = "--")
ax1.plot(forestthermo["Temperature"].sel(z = slice(0,2000))-pasturethermo["Temperature"].sel(z = slice(0,2000)), forestthermo["z"].sel(z = slice(0, 2000)))
ax1.set_xlabel("Altitude (m)")
ax1.set_xlabel("Temperature (K)")
ax2.plot(forestthermo["VaporMix"].sel(z = slice(0,2000))-pasturethermo["VaporMix"].sel(z = slice(0, 2000)), forestthermo["z"].sel(z = slice(0, 2000)), color = "dodgerblue")
ax2.set_xlabel("Vapor Mixing Ratio (g/kg")
fig.savefig("../PaperFigs/profdif.png")
# plt.plot(forestthermo["z"].sel(z = slice(0, 2000)), pasturethermo["Temperature"])

## Calculations of Surface Conditions

In [ ]:
print(afile_coniforest_noon["CanopyVaporMix"].isel(patch = 1, y = slice(500,800)).mean())
print(afile_pasture_noon["CanopyVaporMix"].isel(patch = 1, y = slice(500, 800)).mean())

In [ ]:
plt.plot(afile_pastureconiforest_noon["SrfPres"].isel(y = slice(500, 800)).mean(dim = "y"))
plt.plot(afile_suburbconiforest_noon["SrfPres"].isel(y = slice(500, 800)).mean(dim = "y"))

In [ ]:
plt.figure(figsize = (4, 4), dpi = 200)
plt.plot(afile_pastureconiforest_noon["ShortwaveDownSrf"].isel(y=500)*(1-afile_coniforest_noon["SurfaceAlbedo"].isel(y=500)) - afile_pastureconiforest_noon["LongwaveUpSrf"].isel(y=500) + afile_pastureconiforest_noon["LongwaveDownSrf"].isel(y = 500), color = "gold")
# plt.plot(afile_pastureconiforest_noon["ShortwaveDownSrf"].isel(y = 500), color = "chartreuse")
plt.plot(afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = 500), color = "red")
plt.plot(afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = 500), color = "purple")
plt.plot(afile_pastureconiforest_noon["ShortwaveDownSrf"].isel(y=500)*(1-afile_coniforest_noon["SurfaceAlbedo"].isel(y=500)) - afile_pastureconiforest_noon["LongwaveUpSrf"].isel(y=500) + afile_pastureconiforest_noon["LongwaveDownSrf"].isel(y = 500) - afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = 500) - afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = 500), color = "black", linestyle = "--")

In [ ]:
plt.figure(figsize = (4, 4), dpi = 200)
plt.plot(afile_suburbconiforest_noon["ShortwaveDownSrf"].isel(y=500)*(1-afile_coniforest_noon["SurfaceAlbedo"].isel(y=500)) - afile_suburbconiforest_noon["LongwaveUpSrf"].isel(y=500) + afile_suburbconiforest_noon["LongwaveDownSrf"].isel(y = 500), color = "gold")
# plt.plot(afile_suburbconiforest_noon["ShortwaveDownSrf"].isel(y = 500), color = "chartreuse")
plt.plot(afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = 500), color = "red")
plt.plot(afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = 500), color = "purple")
plt.plot(afile_suburbconiforest_noon["ShortwaveDownSrf"].isel(y=500)*(1-afile_coniforest_noon["SurfaceAlbedo"].isel(y=500)) - afile_suburbconiforest_noon["LongwaveUpSrf"].isel(y=500) + afile_suburbconiforest_noon["LongwaveDownSrf"].isel(y = 500) - afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = 500) - afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = 500), color = "black", linestyle = "--")

In [ ]:
plt.figure(figsize = (4, 4), dpi = 200)
plt.plot(afile_suburbpasture_noon["ShortwaveDownSrf"].isel(y=500)*(1-afile_pasture_noon["SurfaceAlbedo"].isel(y=500)) - afile_suburbpasture_noon["LongwaveUpSrf"].isel(y=500) + afile_suburbpasture_noon["LongwaveDownSrf"].isel(y = 500), color = "gold")
# plt.plot(afile_suburbpasture_noon["ShortwaveDownSrf"].isel(y = 500), color = "chartreuse")
plt.plot(afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = 500), color = "red")
plt.plot(afile_suburbpasture_noon["LatentHeatFlux"].isel(y = 500), color = "purple")
plt.plot(afile_suburbpasture_noon["ShortwaveDownSrf"].isel(y=500)*(1-afile_pasture_noon["SurfaceAlbedo"].isel(y=500)) - afile_suburbpasture_noon["LongwaveUpSrf"].isel(y=500) + afile_suburbpasture_noon["LongwaveDownSrf"].isel(y = 500) - afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = 500) - afile_suburbpasture_noon["LatentHeatFlux"].isel(y = 500), color = "black", linestyle = "--")

In [ ]:
print(afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(0.5*afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean())
print(afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(0.5*afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = slice(500, 800), x = slice(0, 50)).mean()+0.5*afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = slice(500, 800), x = slice(350, 400)).mean())
print(afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(0.5*afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean())
print(afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(0.5*afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = slice(500, 800), x = slice(0, 50)).mean()+0.5*afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = slice(500, 800), x = slice(350, 400)).mean())
print(afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean()/afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print((0.5*afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_pastureconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean())/(0.5*afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_pastureconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean()))
print(afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean()/afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print((0.5*afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_suburbconiforest_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean())/(0.5*afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_suburbconiforest_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean()))

In [ ]:
print(afile_pastureconiforest_noon["SrfPres"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(0.5*afile_pastureconiforest_noon["SrfPres"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_pastureconiforest_noon["SrfPres"].isel(y = slice(500,800), x = slice(350, 400)).mean())
print(afile_suburbconiforest_noon["SrfPres"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(0.5*afile_suburbconiforest_noon["SrfPres"].isel(y = slice(500, 800), x = slice(0, 50)).mean()+0.5*afile_suburbconiforest_noon["SrfPres"].isel(y = slice(500, 800), x = slice(350, 400)).mean())

In [ ]:
print(afile_pastureconiforest_noon["Pressure"].isel(z = 0, y = slice(500,800), x = slice(150, 250)).mean())
print(0.5*afile_pastureconiforest_noon["Pressure"].isel(z = 0, y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_pastureconiforest_noon["Pressure"].isel(z = 0, y = slice(500,800), x = slice(350, 400)).mean())
print(afile_suburbconiforest_noon["Pressure"].isel(z = 0, y = slice(500, 800), x = slice(150, 250)).mean())
print(0.5*afile_suburbconiforest_noon["Pressure"].isel(z = 0, y = slice(500, 800), x = slice(0, 50)).mean()+0.5*afile_suburbconiforest_noon["Pressure"].isel(z = 0, y = slice(500, 800), x = slice(350, 400)).mean())

In [ ]:
print(afile_pastureconiforest_noon["z"])

In [ ]:
print(afile_suburbpasture_noon["LatentHeatFlux"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(0.5*afile_suburbpasture_noon["LatentHeatFlux"].isel(y = slice(500, 800), x = slice(0, 50)).mean()+0.5*afile_suburbpasture_noon["LatentHeatFlux"].isel(y = slice(500, 800), x = slice(350, 450)).mean())
print(afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(0.5*afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean())
print(afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean()/afile_suburbpasture_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print((0.5*afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_suburbpasture_noon["SensibleHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean())/(0.5*afile_suburbpasture_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(0, 50)).mean()+0.5*afile_suburbpasture_noon["LatentHeatFlux"].isel(y = slice(500,800), x = slice(350, 400)).mean()))

In [ ]:
print(afile_suburb_noon["LongwaveUpSrf"].isel(y = slice(500,800)).mean())
print(afile_suburb_noon["VegTemp"].isel(patch = 1, y = slice(500, 800)).mean())
print(afile_pasture_noon["LongwaveUpSrf"].isel(y = slice(500, 800)).mean())
print(afile_pasture_noon["VegTemp"].isel(patch = 1, y = slice(500, 800)).mean())
print(afile_coniforest_noon["LongwaveUpSrf"].isel(y = slice(500,800)).mean())
print(afile_coniforest_noon["VegTemp"].isel(patch = 1, y = slice(500, 800)).mean())

In [ ]:
print(afile_suburbpasture_noon["SrfTemp"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(0.5*afile_suburbpasture_noon["SrfTemp"].isel(y = slice(500, 800), x = slice(0, 50)).mean()+0.5*afile_suburbpasture_noon["SrfTemp"].isel(y = slice(500, 800), x = slice(350, 450)).mean())

In [ ]:
print(afile_pastureconiforest_noon["SoilEnergy"].isel(patch = 1, y = slice(500,800), x = slice(150, 250)).integrate(coord = "soillev").mean())
print(afile_pastureconiforest_noon["SoilEnergy"].isel(patch = 1, y = slice(500,800), x = slice(0, 50)).integrate(coord = "soillev").mean())

In [ ]:
print(afile_pastureconiforest_noon["VegTemp"].isel(patch = 1, y = slice(500,800), x = slice(150, 250)).mean())
print(afile_pastureconiforest_noon["VegTemp"].isel(patch = 1, y = slice(500,800), x = slice(0, 50)).mean())

In [ ]:
print(afile_pastureconiforest_noon["SrfTemp"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(afile_pastureconiforest_noon["SrfTemp"].isel(y = slice(500,800), x = slice(0, 50)).mean())
print(afile_suburbconiforest_noon["SrfTemp"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(afile_suburbconiforest_noon["SrfTemp"].isel(y = slice(500,800), x = slice(0, 50)).mean())

print(afile_pastureconiforest_noon["SrfPres"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(afile_pastureconiforest_noon["SrfPres"].isel(y = slice(500,800), x = slice(0, 50)).mean())
print(afile_suburbconiforest_noon["SrfPres"].isel(y = slice(500,800), x = slice(150, 250)).mean())
print(afile_suburbconiforest_noon["SrfPres"].isel(y = slice(500,800), x = slice(0, 50)).mean())
print(afile_suburbpasture_noon["SrfPres"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(afile_suburbpasture_noon["SrfPres"].isel(y = slice(500, 800), x = slice(0, 50)).mean())

In [ ]:
print(afile_pastureconiforest_noon["SoilEnergy"].isel(patch = 1, y = slice(500,800), x = slice(150, 250)).integrate(coord = "soillev").mean())
print(afile_pastureconiforest_noon["SoilEnergy"].isel(patch = 1, y = slice(500,800), x = slice(0, 50)).integrate(coord = "soillev").mean())

## PWAT Calculations

In [ ]:
print(afile_pastureconiforest_noon["PWAT"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(afile_pastureconiforest_noon["PWAT"].isel(y = slice(500, 800), x = slice(0, 50)).mean())
print(afile_suburbconiforest_noon["PWAT"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(afile_suburbconiforest_noon["PWAT"].isel(y = slice(500, 800), x = slice(0, 50)).mean())
print(afile_suburbpasture_noon["PWAT"].isel(y = slice(500, 800), x = slice(150, 250)).mean())
print(afile_suburbpasture_noon["PWAT"].isel(y = slice(500, 800), x = slice(0, 50)).mean())

## Print Some Surface Calculations more Readably

In [ ]:
print("PCF Forest 200 m Vapor Mixing Ratio", afile_pastureconiforest_noon["VaporMix"].isel(z = 5, y = slice(500, 800), x = slice(150, 250)).mean().values)
print("PCF Pasture 200 m Vapor Mixing Ratio", afile_pastureconiforest_noon["VaporMix"].isel(z = 5, y = slice(500, 800), x = slice(0, 50)).mean().values)
print("SCF Forest 200 m Vapor Mixing Ratio", afile_suburbconiforest_noon["VaporMix"].isel(z = 5, y = slice(500, 800), x = slice(150, 250)).mean().values)
print("PCF Suburb 200 m Vapor Mixing Ratio", afile_suburbconiforest_noon["VaporMix"].isel(z = 5, y = slice(500, 800), x = slice(0, 50)).mean().values)
print("SP Pasture 200 m Vapor Mixing Ratio", afile_suburbpasture_noon["VaporMix"].isel(z = 5, y = slice(500, 800), x = slice(150, 250)).mean().values)
print("SP Suburb 200 m Vapor Mixing Ratio",afile_suburbpasture_noon["VaporMix"].isel(z = 5, y = slice(500, 800), x = slice(0, 50)).mean().values)

print("PCF Forest 200 m Air Temperature", afile_pastureconiforest_noon["Temperature"].isel(z = 5, y = slice(500, 800), x = slice(150, 250)).mean().values)
print("PCF Pasture 200 m Air Temperature", afile_pastureconiforest_noon["Temperature"].isel(z = 5, y = slice(500, 800), x = slice(0, 50)).mean().values)
print("SCF Forest 200 m Air Temperature", afile_suburbconiforest_noon["Temperature"].isel(z = 5, y = slice(500, 800), x = slice(150, 250)).mean().values)
print("SCF Suburb 200 m Air Temperature", afile_suburbconiforest_noon["Temperature"].isel(z = 5, y = slice(500, 800), x = slice(0, 50)).mean().values)
print("SP Pasture 200 m Air Temperature", afile_suburbpasture_noon["Temperature"].isel(z = 5, y = slice(500, 800), x = slice(150, 250)).mean().values)
print("SP Suburb 200 m Air Temperature", afile_suburbpasture_noon["Temperature"].isel(z = 5, y = slice(500, 800), x = slice(0, 50)).mean().values)

In [ ]:
print(afile_pastureconiforest_noon["VaporMix"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z = 1500, method = "nearest").mean())
print(afile_pastureconiforest_noon["VaporMix"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbconiforest_noon["VaporMix"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbconiforest_noon["VaporMix"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbpasture_noon["VaporMix"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbpasture_noon["VaporMix"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z = 1500, method = "nearest").mean())

In [ ]:
print(afile_pastureconiforest_noon["Temperature"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z = 1500, method = "nearest").mean())
print(afile_pastureconiforest_noon["Temperature"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbconiforest_noon["Temperature"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbconiforest_noon["Temperature"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbpasture_noon["Temperature"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z = 1500, method = "nearest").mean())
print(afile_suburbpasture_noon["Temperature"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z = 1500, method = "nearest").mean())

In [ ]:
plt.pcolormesh(afile_suburbpasture_noon["x"], afile_suburbpasture_noon["z"].sel(z=zbnds), afile_suburbpasture_noon["Temperature"].isel(y = slice(500, 800)).sel(z = zbnds).mean(dim = "y") - afile_pasture_noon["Temperature"].isel(y = slice(500, 800)).sel(z = zbnds).mean(dim = "y"), vmin = -2, vmax = 2, cmap = "Spectral_r")

In [ ]:
plt.pcolormesh(afile_pastureconiforest_noon["w"].sel(z = 2000, method = "nearest"), vmin = -5, vmax = 5, cmap = "bwr")

In [ ]:
plt.pcolormesh(afile_pastureconiforest_noon["x"], afile_pastureconiforest_noon["z"].sel(z=zbnds), afile_pastureconiforest_noon["Dewpoint"].isel(y = slice(500, 800)).sel(z = zbnds).mean(dim = "y") - afile_pasture_noon["Dewpoint"].isel(y = slice(500, 800)).sel(z = zbnds).mean(dim = "y"), vmin = -2, vmax = 2, cmap = "BrBG")

## Evaluate Surface Temperature and Pressure at Different Times

In [ ]:
ftime = "2022-06-17-230000"
afile_pasture_ftime = xr.open_dataset(f"/moonbow/ascheb/idealized/unipasture-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_suburb_ftime = xr.open_dataset(f"/moonbow/ascheb/idealized/unisuburb-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_coniforest_ftime = xr.open_dataset(f"/moonbow/ascheb/idealized/coniforest-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_pastureconiforest_ftime = xr.open_dataset(f"/moonbow/ascheb/idealized/pastureconiforest-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_suburbconiforest_ftime = xr.open_dataset(f"/moonbow/ascheb/idealized/suburbconiforest-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")
afile_suburbpasture_ftime = xr.open_dataset(f"/moonbow/ascheb/idealized/suburbpasture-wind/ppfiles_paper/mvars-cart-{ftime}-g1.nc")

In [ ]:
from palettable.cartocolors.sequential import Emrld_7
cloudbasecmap = Emrld_7.get_mpl_colormap().reversed()
pfig, pax = plt.subplots(1,1, dpi = 300)
pax.set_facecolor("black")
cldbasemp = pax.pcolormesh(afile_pastureconiforest_ftime["CloudBaseHeight"].isel(y = slice(200,800)).where(afile_pastureconiforest_ftime["CloudBaseHeight"].isel(y = slice(200, 800))<=2500), vmin = 1250, vmax = 2500, cmap = cloudbasecmap)
pfig.colorbar(cldbasemp, orientation = "horizontal")

In [ ]:
print(afile_pastureconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(150, 250)).mean().values)
print(0.5*afile_pastureconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(0, 50)).mean().values + 0.5*afile_pastureconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(350, 400)).mean().values)
print(afile_suburbconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(150, 250)).mean().values)
print(0.5*afile_suburbconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(0, 50)).mean().values + 0.5*afile_suburbconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(350, 400)).mean().values)

In [ ]:
print(afile_pastureconiforest_ftime["SrfPres"].isel(y = slice(500, 800), x = slice(150, 250)).mean().values)
print(0.5*afile_pastureconiforest_ftime["SrfPres"].isel(y = slice(500, 800), x = slice(0, 50)).mean().values + 0.5*afile_pastureconiforest_ftime["SrfPres"].isel(y = slice(500, 800), x = slice(350, 400)).mean().values)
print(afile_suburbconiforest_ftime["SrfPres"].isel(y = slice(500, 800), x = slice(150, 250)).mean().values)
print(0.5*afile_suburbconiforest_ftime["SrfPres"].isel(y = slice(500, 800), x = slice(0, 50)).mean().values + 0.5*afile_suburbconiforest_ftime["SrfPres"].isel(y = slice(500, 800), x = slice(350, 400)).mean().values)

In [ ]:
preszalt = 3000
print(afile_pastureconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z=preszalt, method = "nearest").mean().values)
print(0.5*afile_pastureconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z=preszalt, method = "nearest").mean().values + 0.5*afile_pastureconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(350, 400)).sel(z=preszalt, method = "nearest").mean().values)
print(afile_suburbconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(150, 250)).sel(z=preszalt, method = "nearest").mean().values)
print(0.5*afile_suburbconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(0, 50)).sel(z=preszalt, method = "nearest").mean().values + 0.5*afile_suburbconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(350, 400)).sel(z=preszalt, method = "nearest").mean().values)

In [ ]:
print(afile_pastureconiforest_ftime["SrfTemp"].isel(y = slice(400, 500), x = slice(150, 250)).mean().values)
print(0.5*afile_pastureconiforest_ftime["SrfTemp"].isel(y = slice(400, 500), x = slice(0, 50)).mean().values + 0.5*afile_pastureconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(350, 400)).mean().values)
print(afile_suburbconiforest_ftime["SrfTemp"].isel(y = slice(400, 500), x = slice(150, 250)).mean().values)
print(0.5*afile_suburbconiforest_ftime["SrfTemp"].isel(y = slice(400, 500), x = slice(0, 50)).mean().values + 0.5*afile_suburbconiforest_ftime["SrfTemp"].isel(y = slice(500, 800), x = slice(350, 400)).mean().values)

In [ ]:
print(afile_pastureconiforest_ftime["SrfPres"].isel(y = slice(400, 500), x = slice(150, 250)).mean().values)
print(0.5*afile_pastureconiforest_ftime["SrfPres"].isel(y = slice(400, 500), x = slice(0, 50)).mean().values + 0.5*afile_pastureconiforest_ftime["SrfPres"].isel(y = slice(400, 500), x = slice(350, 400)).mean().values)
print(afile_suburbconiforest_ftime["SrfPres"].isel(y = slice(400, 500), x = slice(150, 250)).mean().values)
print(0.5*afile_suburbconiforest_ftime["SrfPres"].isel(y = slice(400, 500), x = slice(0, 50)).mean().values + 0.5*afile_suburbconiforest_ftime["SrfPres"].isel(y = slice(400, 500), x = slice(350, 400)).mean().values)

In [ ]:
plt.figure(figsize = (4,4), dpi = 300)
plt.plot(afile_pastureconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "red")
plt.plot(0.5*afile_pastureconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("x","y")).values + 0.5*afile_pastureconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "maroon")
plt.plot(afile_suburbconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "red", linestyle = "--")
plt.plot(0.5*afile_suburbconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("x","y")).values + 0.5*afile_suburbconiforest_ftime["Pressure"].isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "maroon", linestyle = "--")

In [ ]:
plt.figure(figsize = (4,4), dpi = 300)
plt.plot((afile_pastureconiforest_ftime["Rho"]*287*afile_pastureconiforest_ftime["Temperature"]).isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "red")
plt.plot(0.5*(afile_pastureconiforest_ftime["Rho"]*287*afile_pastureconiforest_ftime["Temperature"]).isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("x","y")).values + 0.5*(afile_pastureconiforest_ftime["Rho"]*287*afile_pastureconiforest_ftime["Temperature"]).isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "maroon")
plt.plot((afile_suburbconiforest_ftime["Rho"]*287*afile_suburbconiforest_ftime["Temperature"]).isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "red", linestyle = "--")
plt.plot(0.5*(afile_suburbconiforest_ftime["Rho"]*287*afile_suburbconiforest_ftime["Temperature"]).isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("x","y")).values + 0.5*(afile_suburbconiforest_ftime["Rho"]*287*afile_suburbconiforest_ftime["Temperature"]).isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("x","y")).values, afile_pastureconiforest_ftime["z"].values, color = "maroon", linestyle = "--")

In [ ]:
forestthermo_ftime = afile_pastureconiforest_ftime[["Pressure", "Temperature", "Dewpoint"]].isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("y", "x"))
pasturethermo_ftime = (0.5*afile_pastureconiforest_ftime[["Pressure", "Temperature", "Dewpoint"]].isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("y", "x"))+0.5*afile_pastureconiforest_ftime[["Pressure", "Temperature", "Dewpoint"]].isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("y", "x")))
forestthermo_sf_ftime = afile_suburbconiforest_ftime[["Pressure", "Temperature", "Dewpoint"]].isel(y = slice(500, 800), x = slice(150, 250)).mean(dim = ("y", "x"))
suburbthermo_ftime = (0.5*afile_suburbconiforest_ftime[["Pressure", "Temperature", "Dewpoint"]].isel(y = slice(500, 800), x = slice(0, 50)).mean(dim = ("y", "x"))+0.5*afile_suburbconiforest_ftime[["Pressure", "Temperature", "Dewpoint"]].isel(y = slice(500, 800), x = slice(350, 400)).mean(dim = ("y", "x"))) 

import metpy.calc as mcalc
from metpy.plots import SkewT
from metpy.units import units

tfig = plt.figure(figsize = (5, 4), dpi = 200)
testskew = SkewT(tfig, rotation = 45)
testskew.ax.set_ylim(1050, 200)
testskew.ax.set_xlim(10, 40)
testskew.ax.axhline(forestthermo_ftime["Pressure"].sel(z = 200, method = "nearest"), color = "black")
fortemp, = testskew.plot(forestthermo_ftime["Pressure"]*units.hPa, forestthermo_ftime["Temperature"]*units.K-273.15*units.K, color = "red", label = "Temperature over Forest", linewidth = 1)
pastemp, = testskew.plot(pasturethermo_ftime["Pressure"]*units.hPa, pasturethermo_ftime["Temperature"]*units.K-273.15*units.K, color = "maroon", label = "Temperature over Pasture", linewidth = 1)
fordew, = testskew.plot(forestthermo_ftime["Pressure"]*units.hPa, forestthermo_ftime["Dewpoint"]*units.K-273.15*units.K, color = "dodgerblue", label = "Dewpoint over Forest", linewidth = 1)
pasdew, = testskew.plot(pasturethermo_ftime["Pressure"]*units.hPa, pasturethermo_ftime["Dewpoint"]*units.K-273.15*units.K, color = "midnightblue", label = "Dewpoint over Pasture", linewidth = 1)
testskew.plot(forestthermo_sf_ftime["Pressure"]*units.hPa, forestthermo_sf_ftime["Temperature"]*units.K-273.15*units.K, color  = "red", linestyle = "--")
testskew.plot(forestthermo_sf_ftime["Pressure"]*units.hPa, forestthermo_sf_ftime["Dewpoint"]*units.K-273.15*units.K, color = "dodgerblue", linestyle = "--")
testskew.plot(suburbthermo_ftime["Pressure"]*units.hPa, suburbthermo_ftime["Temperature"]*units.K-273.15*units.K, color  = "maroon", linestyle = "--")
testskew.plot(suburbthermo_ftime["Pressure"]*units.hPa, suburbthermo_ftime["Dewpoint"]*units.K-273.15*units.K, color = "midnightblue", linestyle = "--")
dryads = testskew.plot_dry_adiabats(t0 = np.arange(-40, 200, 10)*units.degC, colors = "tomato", linewidths = 0.75, label = "Dry Adiabats", alpha = 0.5)
moistads = testskew.plot_moist_adiabats(t0 = np.arange(-40, 45, 5)*units.degC, colors = "dodgerblue", linewidths = 0.75, label = "Moist Adiabats", alpha = 0.5)
# testskew.plot_mixing_lines(pressure = np.arange(1025, 450, -25)*units.hPa, colors = "limegreen", linewidths = 0.75, label = ")
proflegend = testskew.ax.legend(framealpha = 0, fontsize = 6, loc = "upper right", handles = [fortemp, pastemp, fordew, pasdew])
testskew.ax.add_artist(proflegend)
backlegend = testskew.ax.legend(framealpha = 0, fontsize = 6, loc = "lower left", handles = [dryads, moistads], bbox_to_anchor=(0.0, -0.01))
testskew.ax.add_artist(backlegend)
testskew.ax.set_ylabel("Pressure (hPa)", fontsize = 8)
testskew.ax.set_xlabel("Temperature (C)", fontsize = 8)
testskew.ax.set_title(f"Forest and Pasture Thermodynamic Profiles at {(datetime.strptime(ftime, '%Y-%m-%d-%H%M%S')-timedelta(hours=5)).strftime('%H%M')} LT", fontsize = 10)

In [ ]:
afile_pasture_ftime.close(); del afile_pasture_ftime
afile_coniforest_ftime.close(); del afile_coniforest_ftime
afile_suburb_ftime.close(); del afile_suburb_ftime
afile_pastureconiforest_ftime.close(); del afile_pastureconiforest_ftime
afile_suburbconiforest_ftime.close(); del afile_suburbconiforest_ftime
afile_suburbpasture_ftime.close(); del afile_suburbpasture_ftime

In [ ]:
# plt.plot(afile_pastureconiforest_noon["x"], afile_pastureconiforest_noon["w"].isel(y = slice(500, 800)).rolling(x = 5).mean().sel(z = 1500, method = "nearest").mean(dim = "y"))

In [ ]:
# zbnds = slice(0, 5000)
# plt.figure(figsize = (4,4), dpi = 300)
# plt.yticks([0, 900, 1000, 1100, 1200, 1300, 2000, 3000, 4000])
# plt.pcolormesh(afile_pastureconiforest_noon["x"], afile_pastureconiforest_noon["z"].sel(z = zbnds), afile_pastureconiforest_noon["VaporMix"].sel(z = zbnds).isel(y = slice(500, 800)).mean(dim = "y"), vmin = 4, vmax = 18, cmap = "BrBG")
# # plt.contour(afile_pastureconiforest_noon["x"], afile_pastureconiforest_noon["z"].sel(z=zbnds), afile_pastureconiforest_noon["Pressure"].sel(z=zbnds).isel(y = slice(500, 800)).mean(dim = "y"), colors = "black", linestyles = "--")
# plt.plot(afile_pastureconiforest_noon["x"], 1000*(afile_pastureconiforest_noon["Temperature"].isel(y = slice(500, 800)).sel(z = slice(0, 1000)).mean(dim = ("y", "z"))-afile_pastureconiforest_noon["Dewpoint"].isel(y = slice(500, 800)).sel(z = slice(0, 1000)).mean(dim = ("y", "z")))/8, color = "red", linestyle = "--")
# plt.plot(afile_pastureconiforest_noon["x"], afile_pastureconiforest_noon["CloudBaseHeight"].isel(y = slice(500, 800)).mean(dim = "y"), color = "blue", linestyle = "-.")

## Very Crude Estimate of PBL Height in PF Simulation

In [ ]:
# tempdiff_up = 0
# tempdiff_us = 0
# tempdiff_ucf = 0
# for zlev in afile_pasture_noon["z"][40:0:-1]:
#     # print(zlev)
#     zdiff_up = afile_pasture_noon["VaporMix"][:41,500:,:].mean(dim = ("y","x")).differentiate(coord = "z").differentiate(coord="z").sel(z=zlev)
#     zdiff_us = afile_suburb_noon["VaporMix"][:41,500:,:].mean(dim = ("y","x")).differentiate(coord = "z").differentiate(coord="z").sel(z=zlev)
#     zdiff_ucf = afile_coniforest_noon["VaporMix"][:41,500:,:].mean(dim = ("y","x")).differentiate(coord = "z").differentiate(coord="z").sel(z=zlev)
#     if zdiff_up < tempdiff_up:
#         zpbl_up = zlev
#         tempdiff_up = zdiff_up
#     if zdiff_us < tempdiff_us:
#         zpbl_us = zlev
#         tempdiff_us = zdiff_us
#     if zdiff_ucf < tempdiff_ucf:
#         zpbl_ucf = zlev
#         tempdiff_ucf = zdiff_ucf

# print(zpbl_up)
# print(zpbl_us)
# print(zpbl_ucf)

In [ ]:
# plt.figure(figsize = (4, 4), dpi = 200)
# plt.ylim(0, 3000)
# plt.xlim(7, 18)
# plt.plot(afile_pasture_noon["VaporMix"].isel(y = slice(500,800)).mean(dim = ("x", "y")), afile_pasture_noon["z"], color = "gold")
# plt.plot(afile_suburb_noon["VaporMix"].isel(y = slice(500,800)).mean(dim = ("x", "y")), afile_pasture_noon["z"], color = "maroon")
# plt.plot(afile_coniforest_noon["VaporMix"].isel(y = slice(500,800)).mean(dim = ("x", "y")), afile_pasture_noon["z"], color = "forestgreen")
# # plt.axhline(1700, color = "black")
# plt.axhline(zpbl_up, color = "gold", linestyle = "--")
# plt.axhline(zpbl_us, color = "maroon", linestyle = "--")
# plt.axhline(zpbl_ucf, color = "forestgreen", linestyle = "--")

In [ ]:
print(afile_pastureforest_xc["z"][35])

In [ ]:
vapmp = plt.pcolormesh(afile_pastureforest_xc["VaporMix"].isel(y = slice(500, 700)).mean(dim = "y")[:40,:], vmin = 5, vmax = 20, cmap = "BrBG")

## Plot Forest Breeze Vertical Cross-Section at 1200 LT

In [ ]:
from astropy.convolution import convolve
xctime = datetime.strptime("2022-06-17-170000", "%Y-%m-%d-%H%M%S")
afile_pastureforest_xc = xr.open_dataset(f"/moonbow/ascheb/idealized/pastureconiforest-wind/processed_data/mergedvars_{xctime.strftime('%Y-%m-%d-%H%M%S')}.nc")
convuwind = np.zeros((40, len(afile_pastureforest_xc["x"])))
convwwind = np.zeros((40, len(afile_pastureforest_xc["x"])))
# trikern1d = np.array([0,1,2,3,4,5,6,7,8,9,10,9,8,7,6,5,4,3,2,1,0])
trikern1d = np.ones(21)
print(len(trikern1d))
xstep = 10
zstep = 3
for zlev in range(0, 40):
    convuwind[zlev,:] = convolve(afile_pastureforest_xc["u"].isel(z = zlev, y = slice(500,700)).mean(dim = "y"), kernel = trikern1d, boundary = "extend", normalize_kernel = True)
    convwwind[zlev,:] = convolve(afile_pastureforest_xc["w"].isel(z = zlev, y = slice(500,700)).mean(dim = "y"), kernel = trikern1d, boundary = "extend", normalize_kernel = True)
fig, ax1 = plt.subplots(1, 1, figsize = (5, 3.33), dpi = 300, layout = "compressed")
ax1.quiver(afile_pastureforest_xc["x"][xstep::xstep]-250, afile_pastureforest_xc["z"][:40:zstep], convuwind[::zstep,xstep::xstep], 5*convwwind[::zstep,xstep::xstep], width = 0.002, scale = 50, zorder = 1, color = "black")
from palettable.lightbartlein.diverging import BlueDarkRed18_11
tempcmap = BlueDarkRed18_11.get_mpl_colormap()
tempmp = ax1.pcolormesh(afile_pastureforest_xc["x"], afile_pastureforest_xc["z"][:40], (afile_pastureforest_xc["Temp"].isel(y = slice(500,700)).mean(dim = "y")-afile_pastureforest_xc["Temp"].isel(y = slice(500,700)).mean(dim = ("x", "y")))[:40,:], vmin = -2, vmax = 2, cmap = tempcmap, zorder = 0)
from palettable.lightbartlein.diverging import GreenMagenta_11
vapcmap = GreenMagenta_11.get_mpl_colormap()
vapxc = ax1.contour(afile_pastureforest_xc["x"], afile_pastureforest_xc["z"][:40], afile_pastureforest_xc["VaporMix"].isel(y = slice(500, 700)).mean(dim = "y")[:40,:], levels = [12, 14], colors = "darkmagenta", linestyles = "--", linewidths = 0.7, zorder = 1)
ax1.clabel(vapxc, levels = [12, 14], inline = True, colors = "black")
ax1.set_ylabel("Altitude (m)", fontsize = 12)
ax1.set_xlabel("Distance From Center (km)", fontsize = 12)
fig.suptitle(f"Temperature and Wind X-Z Cross-Section at {(xctime-timedelta(hours = 5)).strftime('%H%M')} LT", fontsize = 12)
ax1.set_xticks([-100000, -75000, -50000, -25000, 0, 25000, 50000, 75000, 100000], labels = [-100, -75, -50, -25, 0, 25, 50, 75, 100])
ax1.xaxis.set_tick_params(labelsize = 8)
ax1.yaxis.set_tick_params(labelsize = 8)
cbar = fig.colorbar(tempmp, ax = ax1, orientation = "horizontal", fraction = 0.07, extend = "both"); cbar.set_label("Temperature Departure from Domain Mean (K)", fontsize = 10); cbar.ax.tick_params(labelsize = 8)
fig.savefig("/moonbow/ascheb/idealized/PaperFigs/circulationplot_small.png")
plt.close(); del fig; del ax1

In [ ]:
from astropy.convolution import convolve
from matplotlib.lines import Line2D
xctime = datetime.strptime("2022-06-17-170000", "%Y-%m-%d-%H%M%S")
afile_pastureforest_xc = xr.open_dataset(f"/moonbow/ascheb/idealized/pastureconiforest-wind/ppfiles_paper/mvars-cart-{xctime.strftime('%Y-%m-%d-%H%M%S')}-g1.nc")
convuwind = np.zeros((40, len(afile_pastureforest_xc["x"])))
convwwind = np.zeros((40, len(afile_pastureforest_xc["x"])))
# trikern1d = np.array([0,1,2,3,4,5,6,7,8,9,10,9,8,7,6,5,4,3,2,1,0])
trikern1d = np.ones(21)
# print(len(trikern1d))
xstep = 10
zstep = 3
for zlev in range(0, 40):
    convuwind[zlev,:] = convolve(afile_pastureforest_xc["u"].isel(z = zlev, y = slice(500,800)).mean(dim = "y"), kernel = trikern1d, boundary = "extend", normalize_kernel = True)
    convwwind[zlev,:] = convolve(afile_pastureforest_xc["w"].isel(z = zlev, y = slice(500,800)).mean(dim = "y"), kernel = trikern1d, boundary = "extend", normalize_kernel = True)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (5, 7.5), dpi = 300, layout = "compressed")
axlabels = ["(a)", "(b)"]
for i, ax in enumerate(fig.get_axes()):
    ax.set_ylabel("Altitude (m)", fontsize = 12)
    ax.set_xticks([-100000, -75000, -50000, -25000, 0, 25000, 50000, 75000, 100000], labels = [-100, -75, -50, -25, 0, 25, 50, 75, 100])
    ax.xaxis.set_tick_params(labelsize = 8)
    ax.yaxis.set_tick_params(labelsize = 8)
    left, width = 0, 0.04
    bottom, height = 0.92, 0.08
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "whitesmoke")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabels[i], fontsize = 10, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
windmp = ax1.quiver(afile_pastureforest_xc["x"][xstep::xstep]-250, afile_pastureforest_xc["z"][:40:zstep], convuwind[::zstep,xstep::xstep], 5*convwwind[::zstep,xstep::xstep], width = 0.002, scale = 50, zorder = 1, color = "black")
# ax2.quiverkey(windmp, 0.84, 0.92, 5, label = r"5 $m s^{-1}$", labelpos = "E", fontproperties = {"size": 8}, zorder = 3)
from palettable.lightbartlein.diverging import BlueDarkRed18_11
tempcmap = BlueDarkRed18_11.get_mpl_colormap()
tempmp = ax1.pcolormesh(afile_pastureforest_xc["x"], afile_pastureforest_xc["z"][:40], (afile_pastureforest_xc["Temperature"].isel(y = slice(500,800)).mean(dim = "y")-afile_pastureforest_xc["Temperature"].isel(y = slice(500,800)).mean(dim = ("x", "y")))[:40,:], vmin = -2, vmax = 2, cmap = tempcmap, zorder = 0)
fakecontour = Line2D([], [], color = "darkmagenta", linestyle = "--", linewidth = 1, label = r"Water Vapor Mixing Ratio ($g \ kg^{-1}$)")
# leftbound = ax1.axvline(-50000, color = "dimgrey", linewidth = 1, linestyle = "-.", label = "Forest Boundary", zorder = 2)
# rightbound = ax1.axvline(50000, color = "dimgrey", linewidth = 1, linestyle = "-.", zorder = 2)
vapxc = ax1.contour(afile_pastureforest_xc["x"], afile_pastureforest_xc["z"][:40], afile_pastureforest_xc["VaporMix"].isel(y = slice(500, 800)).mean(dim = "y")[:40,:], levels = [12, 14], colors = "darkmagenta", linestyles = "--", linewidths = 0.7, zorder = 1)
ax1.clabel(vapxc, levels = [12, 14], inline = True, colors = "black")
# ax1.set_ylabel("Altitude (m)", fontsize = 12)
# ax1.set_xlabel("Distance From Center (km)", fontsize = 12)
ax1.set_title(f"Temperature and Wind X-Z Cross-Section at {(xctime-timedelta(hours = 5)).strftime('%H%M')} LT", fontsize = 12)
ax1.legend(framealpha = 0.5, handles = [fakecontour], fontsize = 10)
# ax1.set_xticks([-100000, -75000, -50000, -25000, 0, 25000, 50000, 75000, 100000], labels = [-100, -75, -50, -25, 0, 25, 50, 75, 100])
# ax1.xaxis.set_tick_params(labelsize = 8)
# ax1.yaxis.set_tick_params(labelsize = 8)
tempcbar = fig.colorbar(tempmp, ax = ax1, orientation = "horizontal", fraction = 0.07, extend = "both"); tempcbar.set_label("Temperature Departure from Domain Mean (K)", fontsize = 12); tempcbar.ax.tick_params(labelsize = 10)
ax1.tick_params(labelsize = 10)
windmp = ax2.quiver(afile_pastureforest_xc["x"][xstep::xstep]-250, afile_pastureforest_xc["z"][:40:zstep], convuwind[::zstep,xstep::xstep], 5*convwwind[::zstep,xstep::xstep], width = 0.002, scale = 50, zorder = 1, color = "black")
msemp = ax2.pcolormesh(afile_pastureforest_xc["x"], afile_pastureforest_xc["z"][:40], calc_mse(afile_pastureforest_xc.isel(y = slice(500,800)).isel(z = slice(0,40)).mean(dim = "y")), cmap = "Spectral_r", vmin = 330, vmax = 345, zorder = 0)
ax2.quiverkey(windmp, 0.84, 0.92, 5, label = r"5 $m s^{-1}$", labelpos = "E", fontproperties = {"size": 10}, zorder = 3)
# msemp = ax2.pcolormesh(afile_pastureforest_xc["x"], afile_pastureforest_xc["z"][:40], afile_pastureforest_xc["Temperature"].isel(y = slice(500,800), z = slice(0,40)).mean(dim = "y") + 1864/(1004 * 1000) * afile_pastureforest_xc["VaporMix"].isel(y = slice(500, 800), z = slice(0, 40)).mean(dim = "y"), cmap = "Spectral_r", zorder = 0)
# msemp = ax2.pcolormesh(afile_pastureforest_xc["x"], afile_pastureforest_xc["z"][:40], (afile_pastureforest_xc["Temperature"].isel(y = slice(500,800), z = slice(0,40)).mean(dim = "y")  + afile_pastureforest_xc["VaporMix"].isel(y = slice(500,800), z = slice(0, 40)).mean(dim = "y") * 2.5*10**3 / 1004) * (1000/afile_pastureforest_xc["Pressure"].isel(y = slice(500,800), z = slice(0, 40)).mean(dim = "y"))**0.286, cmap = "Spectral_r", zorder = 0)
ax2.set_title(f"MSE and Wind X-Z Cross-Section at {(xctime-timedelta(hours = 5)).strftime('%H%M')} LT", fontsize = 12)
ax2.set_xlabel("Distance From Center (km)", fontsize = 12)
msecbar = fig.colorbar(msemp, ax = ax2, orientation = "horizontal", fraction = 0.07, extend = "both")
msecbar.set_label("Normalized Moist Static Energy (K)", fontsize = 12)
msecbar.ax.set_xticks([330, 335, 340, 345])
msecbar.ax.tick_params(labelsize = 10)
ax2.tick_params(labelsize = 10)
fig.get_layout_engine().set(h_pad = 0.05, hspace = 0.1)
fig.savefig("/moonbow/ascheb/idealized/PaperFigs/circulationplot_2panel.png")
# plt.close(); del fig; del ax1; del ax2

## Comparing Simulations with and Without Background Wind

In [ ]:
# afile_pastureconiforest_wind_end = xr.open_dataset("/moonbow/ascheb/idealized/pastureconiforest-wind/processed_data/mergedvars_2022-06-17-230000.nc")
# afile_pastureconiforest_nowind_end = xr.open_dataset("/moonbow/ascheb/idealized/pastureconiforest-nowind/processed_data/mergedvars_2022-06-17-230000.nc")
# afile_pasture_wind_end = xr.open_dataset("/moonbow/ascheb/idealized/unipasture-wind/processed_data/mergedvars_2022-06-17-230000.nc")
# afile_pasture_nowind_end = xr.open_dataset("/moonbow/ascheb/idealized/unipasture-nowind/processed_data/mergedvars_2022-06-17-230000.nc")

In [ ]:
# # for field in fields:
# fig, ((ax11, ax12), (ax21, ax22)) = plt.subplots(2, 2, figsize = (16.67, 25), dpi = 200, layout = "constrained")
# for ax in fig.get_axes():
#     ax.set_aspect(1)
#     ax.set_xticks([])
#     ax.set_yticks([])
# for ax in (ax11, ax21):
#     ax.set_ylabel("Distance from Shore (km)")
#     ax.set_yticks([200, 300, 450, 600, 800], labels = ["-50 ", "0  ", "75 ", 150, 250])
# for ax in (ax21, ax22):
#     ax.set_xlabel("Distance from Center (km)")
#     ax.set_xticks([0, 100, 200, 300, 400], labels = [-100, -50, 0, 50, 100])
# ax11.set_title("Pasture - Wind", fontfamily = "Liberation Serif", y = 1.02)
# ax12.set_title("Pasture - No Wind", fontfamily = "Liberation Serif", y = 1.02)
# ax21.set_title("Pasture-ENF - Wind", fontfamily = "Liberation Serif", y = 1.02)
# ax22.set_title("Pasture-ENF - No Wind", fontfamily = "Liberation Serif", y = 1.02)
# t = datetime(2022, 6, 17, 23, 0, 0)
# fig.suptitle(f"Accumulated Rainfall as of{(t-timedelta(hours=5)).strftime('%H%M')} LT")
# rainbounds = [0, 0.1, 0.3, 0.5, 1.5, 2.5, 5.5, 7.5, 11.75, 15, 20, 25, 37.5, 50, 70]
# rainnorm = BoundaryNorm(boundaries = rainbounds, ncolors = 256, extend = "max")
# rainmp = ax11.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), afile_pasture_wind_end["RainPrecipTotal"].where(afile_pasture_wind_end["RainPrecipTotal"]>0.01).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
# ax12.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), afile_pasture_nowind_end["RainPrecipTotal"].where(afile_pasture_nowind_end["RainPrecipTotal"]>0.01).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
# ax21.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), afile_pastureconiforest_wind_end["RainPrecipTotal"].where(afile_pastureconiforest_wind_end["RainPrecipTotal"]>0.01).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
# ax22.pcolormesh(np.linspace(0, 400, 401), np.linspace(200, 800, 601), afile_pastureconiforest_nowind_end["RainPrecipTotal"].where(afile_pastureconiforest_nowind_end["RainPrecipTotal"]>0.01).isel(y = slice(200,800)), cmap = "jet", norm = rainnorm)
# for ax in fig.get_axes():
#     ax.axhline(300, color = "white", linestyle = "--", label = "shoreline", linewidth = 2)
#     ax.legend(loc = "lower left", fontsize = 30)
# cbar = fig.colorbar(rainmp, ax = fig.get_axes(), orientation = "horizontal", fraction = 0.05, extend = "max"); cbar.set_label("Rain Rate (mm/hr)")
# fig.savefig(f"/moonbow/ascheb/idealized/combplots/raintotal_windcomp.png")

In [ ]:
# print(afile_pastureforest_xc.variables)